In [32]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('train.csv', delimiter=';')
df_tuesday = df[df['Publication_Day'] == 'Tuesday']

 Kalma Filter 

In [ ]:
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
data_col = 'Listening_Time_minutes'
data = df_tuesday[data_col].dropna().values

# Khởi tạo Basic Kalman Filter
kf = KalmanFilter(dim_x=2, dim_z=1)
kf.x = np.array([data[0], 0.])  # giá trị ban đầu, tốc độ
kf.F = np.array([[1., 1.], [0., 1.]])  # ma trận chuyển đổi trạng thái
kf.H = np.array([[1., 0.]])           # ma trận đo
kf.P *= 1000.                         # hiệp phương sai ban đầu
kf.R = 10                             # nhiễu đo
kf.Q = Q_discrete_white_noise(dim=2, dt=1, var=0.01)  # Nhiễu hệ thống

# Lọc tuần tự
filtered = []
for z in data:
    kf.predict()
    kf.update(z)
    filtered.append(kf.x[0])

plt.figure(figsize=(12, 6))
plt.plot(data, label='Dữ liệu gốc', alpha=0.6)
plt.plot(filtered, label='Dữ liệu sau Kalman Filter', color='red', linewidth=2)
plt.title('Basic Kalman Filter - Listening_Time_minutes')
plt.xlabel('Thời gian')
plt.ylabel('Listening_Time_minutes')
plt.legend()
plt.grid(True)
plt.tight_layout()
from filterpy.kalman import KalmanFilter, rts_smoother
from filterpy.common import Q_discrete_white_noise

data_col = 'Listening_Time_minutes'
data_values = df_tuesday[data_col].dropna().values

kf = KalmanFilter(dim_x=2, dim_z=1)
kf.x = np.array([data_values[0], 0.])  # [vị trí, vận tốc]
kf.F = np.array([[1., 1.], [0., 1.]])
kf.H = np.array([[1., 0.]])
kf.P *= 1000.
kf.R = 10
kf.Q = Q_discrete_white_noise(dim=2, dt=1, var=0.01)

mu, cov = [], []

for z in data_values:
    kf.predict()
    kf.update(z)
    mu.append(kf.x.copy())
    cov.append(kf.P.copy())

mu = np.array(mu)
cov = np.array(cov)
# Lọc tuần tự
filtered = []
for z in data:
    kf.predict()
    kf.update(z)
    filtered.append(kf.x[0])

plt.figure(figsize=(12, 6))
plt.plot(data, label='Dữ liệu gốc', alpha=0.6)
plt.plot(filtered, label='Dữ liệu sau Kalman Filter', color='red', linewidth=2)
plt.title('Basic Kalman Filter - Listening_Time_minutes (Friday)')
plt.xlabel('Thời gian')
plt.ylabel('Listening_Time_minutes')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
plt.show()


 Smooth 

In [ ]:
# Làm mượt bằng RTS smoother
smoothed_mu, smoothed_cov, _, _ = kf.rts_smoother(mu, cov)

# Vẽ biểu đồ
plt.figure(figsize=(12, 6))
plt.plot(data, label='Dữ liệu gốc', color='gray', linestyle='--', alpha=0.6)
plt.plot(mu[:, 0], label='Kalman Filter', color='blue')
plt.plot(smoothed_mu[:, 0], label='RTS Smooth', color='green')
plt.title('Kalman Filter và RTS Smoother cho Listening Time')
plt.xlabel('Thời gian')
plt.ylabel('Listening Time (minutes)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Khởi tạo bộ lọc Kalman
kf = SimpleKalmanFilter(
    initial_state=data[0],
    initial_covariance=1.0,
    process_covariance=1.0,
    measurement_covariance=5.0
)

# Lọc dữ liệu quan sát
filtered_state = []

# Lọc qua tất cả các dữ liệu quan sát
for z in data:
    kf.predict()  # Tiên đoán giá trị tiếp theo
    kf.update(z)  # Cập nhật với giá trị thực tế
    filtered_state.append(kf.state)

# Dự đoán giá trị tương lai
future_steps = 10  # Dự đoán 10 bước tiếp theo
predictions = []

for _ in range(future_steps):
    kf.predict()  # Tiên đoán bước tiếp theo
    predictions.append(kf.state)

# Vẽ kết quả
plt.figure(figsize=(12, 6))
plt.plot(data, label='Dữ liệu quan sát (Observations)', alpha=0.6)
plt.plot(filtered_state, label='Trạng thái dự đoán (Filtered state)', linewidth=2)
plt.plot(range(len(data), len(data) + future_steps), predictions, label='Dự đoán tương lai (Predictions)', linestyle='dotted', color='orange')
plt.legend()
plt.title("Kalman Filter: Dự đoán trạng thái và giá trị tương lai cho dữ liệu Sunday")
plt.xlabel("Thời gian (Time)")
plt.ylabel("Listening Time (minutes)")
plt.grid(True)
plt.tight_layout()
plt.show()
